In [3]:
%%capture
!pip install --upgrade pip
!git clone https://github.com/josephmargaryan/ATDL2.git
%cd ATDL2
!pip install -e . --no-deps


# Pareto Multi-Objective Bayesian Optimization with Conservative Pruning

This notebook runs Bayesian optimization using:

## Optimization Strategy
- **Multi-objective**: Optimizes CR (Compression Rate) and Accuracy simultaneously
- **Sampler**: NSGA-II (automatic for Pareto mode)
- **Output**: Pareto front of optimal trade-off solutions

## Pruning Strategy: CONSERVATIVE
All experiments use conservative pruning to maximize quality and minimize risk:

```
--enable-pruning
--pruning-warmup-trials 15     # Wait for 30% of trials before pruning
--pruning-warmup-steps 20      # Wait until 50% of epochs before pruning
--pruning-percentile 75        # Only prune worst 25% of trials
--cr-every 5                   # Monitor CR every 5 epochs
```

### Why Conservative?
- **Multi-objective optimization**: Pruning only monitors CR, not accuracy
- **Preserves Pareto diversity**: Allows high-accuracy/moderate-CR solutions to develop
- **Low risk**: Minimizes chance of pruning "late bloomers"
- **Expected speedup**: 1.1-1.3× (10-15% time savings)

### Proportional Adjustments
- **30-trial runs**: Use 9 warmup trials (30%), 15 warmup steps (50% of 30 epochs)

In [ ]:
!python scripts/tune_optuna.py \
    --preset lenet_300_100 \
    --n-trials 50 \
    --study-name lenet300_pareto \
    --use-pareto \
    --enable-pruning \
    --pruning-warmup-trials 15 \
    --pruning-warmup-steps 20 \
    --pruning-percentile 75 \
    --save-dir runs/bayesian_optimization_pareto/lenet_300_100 \
    --timeout-sec 7200 \
    --keep-failed \
    --no-huffman \
    --quant-skip-last \
    --batch-size 128 \
    --num-workers 2 \
    --pretrain-epochs 0 \
    --retrain-epochs 40 \
    --lr-pre 5e-4 \
    --optim-pre adam \
    --eval-every 5 \
    --cr-every 5 \
    --seed 1 \
    --load-pretrained runs/mnist_300/mnist_lenet_300_100_pre.pt

In [ ]:
!python scripts/tune_optuna.py \
    --preset lenet5 \
    --n-trials 50 \
    --study-name lenet5_pareto \
    --use-pareto \
    --enable-pruning \
    --pruning-warmup-trials 15 \
    --pruning-warmup-steps 20 \
    --pruning-percentile 75 \
    --save-dir runs/bayesian_optimization_pareto/lenet5 \
    --timeout-sec 7200 \
    --keep-failed \
    --no-huffman \
    --quant-skip-last \
    --batch-size 128 \
    --num-workers 2 \
    --pretrain-epochs 0 \
    --retrain-epochs 40 \
    --lr-pre 5e-4 \
    --optim-pre adam \
    --eval-every 5 \
    --cr-every 5 \
    --seed 1 \
    --load-pretrained runs/mnist_caffe/mnist_lenet5_pre.pt

In [ ]:
!python scripts/tune_optuna.py \
    --preset wrn_16_4 \
    --n-trials 50 \
    --study-name wrn16_pareto \
    --use-pareto \
    --enable-pruning \
    --pruning-warmup-trials 15 \
    --pruning-warmup-steps 20 \
    --pruning-percentile 75 \
    --save-dir runs/bayesian_optimization_pareto/resnet \
    --timeout-sec 7200 \
    --keep-failed \
    --no-huffman \
    --quant-skip-last \
    --batch-size 128 \
    --num-workers 2 \
    --pretrain-epochs 0 \
    --retrain-epochs 40 \
    --lr-pre 5e-4 \
    --optim-pre adam \
    --eval-every 5 \
    --cr-every 5 \
    --seed 1 \
    --load-pretrained runs/resnet/cifar10_wrn_16_4_quantized.pt

In [ ]:
!python scripts/tune_optuna.py \
  --preset wrn_16_4 \
  --n-trials 30 \
  --study-name wrn16_pareto_run2 \
  --use-pareto \
  --enable-pruning \
  --pruning-warmup-trials 9 \
  --pruning-warmup-steps 15 \
  --pruning-percentile 75 \
  --pretrain-epochs 0 \
  --save-dir runs/bayesian_optimization_pareto/wrn_16_4 \
  --timeout-sec 7200 \
  --no-huffman \
  --keep-failed \
  --quant-skip-last \
  --batch-size 64 \
  --num-workers 1 \
  --retrain-epochs 30 \
  --lr-pre 5e-4 \
  --optim-pre adam \
  --eval-every 5 \
  --cr-every 5 \
  --seed 1 \
  --load-pretrained runs/pt_wrn_16_4_pre/cifar10_wrn_16_4_pre.pt

# Visualize Pareto Front Results

Use these commands to visualize the Pareto-optimal solutions from the optimization runs above.

In [ ]:
# Visualize LeNet-300-100 Pareto front
!python scripts/tune_optuna_pareto_viz.py \
    --pareto-json runs/bayesian_optimization_pareto/lenet_300_100/lenet300_pareto_pareto_results.json \
    --annotate

# Visualize LeNet5 Pareto front
!python scripts/tune_optuna_pareto_viz.py \
    --pareto-json runs/bayesian_optimization_pareto/lenet5/lenet5_pareto_pareto_results.json \
    --annotate

# Visualize WRN-16-4 Pareto front
!python scripts/tune_optuna_pareto_viz.py \
    --pareto-json runs/bayesian_optimization_pareto/resnet/wrn16_pareto_pareto_results.json \
    --annotate